In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import tensorflow as tf


In [ ]:
print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0


In [ ]:
!unzip -q /content/gdrive/MyDrive/CMPE255_ASL_PROJECT/Data.zip

In [ ]:
dataset_path = '/content/asl_alphabet_train/asl_alphabet_train'


In [ ]:
Efficientnet_b0 = 'https://tfhub.dev/google/efficientnet/b0/classification/1'


In [ ]:
import os
import tensorflow_hub as hub
import tensorflow as tf
import pathlib
import cv2
from sklearn.utils import shuffle
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
train_data_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split = 0.2)

test_data_generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2)

In [ ]:
# GPU setup
batch_size = 32
img_height = 224
img_width = 224
train_data = train_data_generator.flow_from_directory(
    dataset_path,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training')

val_data = test_data_generator.flow_from_directory(
    dataset_path,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode='categorical',
    subset = 'validation')

## Efficientnet_b0

In [ ]:
Efficientnet_b0 = 'https://tfhub.dev/google/efficientnet/b0/classification/1'


In [ ]:
model_Efficientnet = tf.keras.Sequential([
    hub.KerasLayer(Efficientnet_b0, input_shape = (224,224,3), name = 'Efficientnet_b0'),

    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(29, activation = 'softmax', name = 'Output_layer')
])



In [ ]:
# model_Efficientnet.trainable = True

In [ ]:
model_Efficientnet.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [ ]:
model_Efficientnet.summary()

In [ ]:
efficientnet_model = model_Efficientnet.fit(train_data, epochs = 25, verbose = 1, validation_data=val_data)


In [ ]:
def plot_history(history, metric, val_metric):
  acc = history.history[metric]
  val_acc = history.history[val_metric]

  loss=history.history['loss']
  val_loss=history.history['val_loss']

  epochs_range = range(len(acc))

  fig=plt.figure(figsize=(12, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.ylim([min(plt.ylim()),1])
  plt.grid(True)
  plt.title('Training and Validation Accuracy')

  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.grid(True)
  plt.title('Training and Validation Loss')
  plt.show()
  fig.savefig('traininghistory.pdf')



In [ ]:
plot_history(efficientnet_model, 'accuracy', 'val_accuracy')

In [ ]:
model_Efficientnet.save('/content/gdrive/MyDrive/CMPE255_ASL_PROJECT/Efficientnet_model_new2.h5')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# model_Efficientnet = load_model('/content/gdrive/MyDrive/CMPE255_ASL_PROJECT/Efficientnet_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
# efficientnet_model = model_Efficientnet.fit(train_data, epochs = 1, verbose = 1, validation_data=val_data)


In [ ]:
# model_Efficientnet = load_model('/content/gdrive/MyDrive/CMPE255_ASL_PROJECT/Efficientnet_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})



## Inception ResNet V2

In [ ]:
inceptionResNetV2 = 'https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/5'

In [ ]:
model_inceptionResNetV2 = tf.keras.Sequential([
    hub.KerasLayer(inceptionResNetV2, input_shape = (224,224,3),
               trainable=True, arguments=dict(batch_norm_momentum=0.997), name = 'inceptionResNetV2_layer'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(29, activation = 'softmax', name = 'Output_layer')
])

model_inceptionResNetV2.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [ ]:
model_inceptionResNetV2.summary()

In [ ]:
inceptionResNetV2_model = model_inceptionResNetV2.fit(train_data, epochs = 25, verbose = 1, validation_data=val_data)


In [ ]:
model_inceptionResNetV2.save('/content/gdrive/MyDrive/CMPE255_ASL_PROJECT/model_inceptionResNetV2.h5')

In [ ]:
plot_history(inceptionResNetV2_model, 'accuracy', 'val_accuracy')